In [ ]:
import os
import sys

sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [ ]:
from langchain.agents import Tool, AgentExecutor, BaseMultiActionAgent, AgentOutputParser
from langchain.agents.chat.base import ChatAgent
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI, VertexAI
from langchain.chat_models import ChatVertexAI, ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain
from typing import List, Union, Optional
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

In [ ]:
!export PROJECT_ID="gcp-mira-develop"
project = "gcp-mira-develop"
os.environ["PROJECT_ID"] = project
PROJECT_ID = project

In [ ]:
import config
from config import (VERTEX_LLM_TYPE_BISON_CHAT,
                    VERTEX_LLM_TYPE_BISON_TEXT,
                    OPENAI_LLM_TYPE_GPT3_5,
                    OPENAI_LLM_TYPE_GPT4,
                    COHERE_LLM_TYPE,
                    LLM_BACKEND_ROBOT_USERNAME,
                    LLM_BACKEND_ROBOT_PASSWORD)
from common.utils.token_handler import UserCredentials
config.TOOLS_SERVICE_BASE_URL = "https://gcp-mira-develop.cloudpssolutions.com/tools-service/api/v1"
config.auth_client = UserCredentials(LLM_BACKEND_ROBOT_USERNAME,
                              LLM_BACKEND_ROBOT_PASSWORD,
                              "https://gcp-mira-develop.cloudpssolutions.com")

In [ ]:
config.SERVICES['tools-service']['host']='https://gcp-mira-develop.cloudpssolutions.com'

In [ ]:
config.SERVICES['tools-service']['api_url_prefix']='https://gcp-mira-develop.cloudpssolutions.com/tools-service/api/v1'

In [ ]:
config.SERVICES['tools-service']

In [ ]:
from common.models import Agent, User
from common.models.agent import AgentType, UserPlan, PlanStep
from services.agents.agent_service import get_agent_config, get_plan_agent_config,agent_execute_plan, run_agent
from services.agents.agent_tools import database_tool, google_sheets_tool

In [ ]:
from services.agents.agents import TaskAgent, PlanAgent
#casey_plan_agent = TaskAgent(VERTEX_LLM_TYPE_BISON_TEXT)
#casey_plan_agent = TaskAgent(VERTEX_LLM_TYPE_BISON_CHAT)
#casey_plan_agent = TaskPlanAgent(OPENAI_LLM_TYPE_GPT3_5)
#casey_plan_agent = TaskAgent(OPENAI_LLM_TYPE_GPT4)
#casey_plan_agent = TaskAgent(OPENAI_LLM_TYPE_GPT4)
#casey_plan_agent = TaskPlanAgent(COHERE_LLM_TYPE)

In [ ]:
#prompt = """Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers"""
prompt = """ Find all FQHCs that have billed an incorrect CDT code and save the results in a Google spreadsheet called FQHC_with_Incorrect_CDTs"""

In [ ]:
#user id for Jon Chen
user_id = "7XqXArrDvknCNWaAOlmS"

In [ ]:
user = User.find_by_email("jonchen@google.com")

In [ ]:
#agent_executor.run(agent_inputs)

In [ ]:
  agent_params = get_agent_config()["Task"]

In [ ]:
agent_params

In [ ]:
llm_service_agent = agent_params["agent_class"](agent_params["llm_type"])

In [ ]:
langchain_agent = llm_service_agent.load_agent()

In [ ]:
type(langchain_agent)

In [ ]:
from services.agents.agent_service import agent_plan, parse_plan

In [ ]:
plan = agent_plan("Plan", prompt, user_id)

user_plan = plan[1]
plan_steps=user_plan.plan_steps
chat_history = []

for step in plan_steps:
    description = PlanStep.find_by_id(step).description
    chat_history.append(description)

In [ ]:
#tools = casey_plan_agent.get_tools()
#print(tools)

#agent = casey_plan_agent.load_agent()

prompt = """You are an AI assistant that can execute steps by calling upon the right tools. You have access to all the information required to execute the plan by from the values returned by the tools ."""
#prompt = """You are an AI assistant that can execute steps by calling upon the right tools. You have access to all the information required to execute the plan by from the values returned by the tools . Execute the steps as outlined in the {task_plan} variable """
#agent_executor = AgentExecutor.from_agent_and_tools(
#  agent=agent, 
#    tools=tools,
#    verbose=True)




In [ ]:
#chat_history[0]= '1. Use [query_tool] to [ find Medicaid applicants missing a pay stub ]'
#chat_history[2] = '3. Use [gmail_tool] to [send the composed email to the identified applicants]' 
#chat_history.insert(1,'Use [docs_tool] to [compose an email asking for a pay stub from their employers]')

In [ ]:
#result = agent_executor.run(agent_inputs)

In [ ]:

#call agent_service that looks up the plan steps for a given User Plan
#agent_execute_plan(agent_name='Task', prompt=agent_inputs,user_plan=user_plan,user=user)
agent_execute_plan(agent_name='Task', prompt=prompt,user_plan=user_plan,user=user)

In [ ]:

#run_agent(agent_name='Task', prompt=agent_inputs['input'])